In [1]:
from TMfunctions import *
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
# Cross Validation
from sklearn.model_selection import KFold

C:\Users\david\AppData\Local\Temp\ipykernel_12680\803022442.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB

In [4]:
from gensim.models import Word2Vec

In [5]:
import numpy as np

# Import Data

In [3]:
train = pd.read_csv('Data/No PreProcessing/train.csv', index_col = 0)
test = pd.read_csv('Data/No PreProcessing/test.csv', index_col = 0)

In [4]:
df = pd.concat([train, test])

In [5]:
# Split the data into sentences
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tot = []
for el in df['medical_abstract']:
    lst = tokenizer.tokenize(el)
    tot += lst

In [9]:
# Save the data
with open("Data/sentences", "wb") as fp:
    pickle.dump(tot, fp)

In [7]:
# Load the data
with open('Data/sentences', 'rb') as f:
    tot = pickle.load(f)

In [8]:
len(tot)

81132

# No Stopwords Removal

In [34]:
# Apply preProcessing
def preprocess(text):
    # Remove punctuation and other non-alphanumeric characters
    text =  re.sub('[^a-zA-Z]', ' ', text)
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    return words
    # Join the words back into a string
    #return ' '.join(words)

prep_tot = [preprocess(x) for x in tot]

In [38]:
model = Word2Vec(prep_tot, min_count=5, vector_size=100, window=7)

In [39]:
train = pd.read_csv('Data/No PreProcessing/train.csv', index_col = 0)
test = pd.read_csv('Data/No PreProcessing/test.csv', index_col = 0)
data = pd.concat([train,test])

In [9]:
import numpy as np
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [41]:
kf = KFold(n_splits=5)

fold = 0

cv = CountVectorizer(binary=True)
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.loc[train_index]
    test = data.loc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['medical_abstract']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['medical_abstract']]))
    
    Kresult = micro_f1(build_results(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB = 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5}
result

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:29<00:00,  5.90s/it]


fold 1:
{'DT': 0.454, 'RF': 0.504, 'NB': 0.555}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:31<00:00,  6.21s/it]


fold 2:
{'DT': 0.471, 'RF': 0.518, 'NB': 0.566}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:34<00:00,  6.83s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:32<00:00,  6.54s/it]


fold 4:
{'DT': 0.454, 'RF': 0.511, 'NB': 0.542}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:32<00:00,  6.51s/it]

fold 5:
{'DT': 0.461, 'RF': 0.519, 'NB': 0.555}
 


{'DT': 0.46299999999999997, 'RF': 0.512, 'NB': 0.5522000000000001}

# StopWords Removal

In [42]:
# Apply preProcessing
def preprocess(text):
    # Remove punctuation and other non-alphanumeric characters
    text =  re.sub('[^a-zA-Z]', ' ', text)
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    # Remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    return words

no_stop = [preprocess(x) for x in tot]

In [43]:
# Save the data
with open("Data/no_stop", "wb") as fp:
    pickle.dump(no_stop, fp)

In [25]:
# Load the data
with open("Data/no_stop", "rb") as fp:
    no_stop = pickle.load(fp)

In [46]:
model = Word2Vec(no_stop, min_count=5, vector_size=100, window=7)

In [47]:
train = pd.read_csv('Data/StopWords/train.csv', index_col = 0)
test = pd.read_csv('Data/StopWords/test.csv', index_col = 0)
data = pd.concat([train,test])

In [48]:
kf = KFold(n_splits=5)

fold = 0

cv = CountVectorizer(binary=True)
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.loc[train_index]
    test = data.loc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['medical_abstract']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['medical_abstract']]))
    
    Kresult = micro_f1(build_results(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB = 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5}
result

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:33<00:00,  6.74s/it]


fold 1:
{'DT': 0.499, 'RF': 0.594, 'NB': 0.569}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:33<00:00,  6.79s/it]


fold 2:
{'DT': 0.54, 'RF': 0.598, 'NB': 0.577}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:35<00:00,  7.07s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:38<00:00,  7.74s/it]


fold 4:
{'DT': 0.512, 'RF': 0.597, 'NB': 0.578}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:35<00:00,  7.11s/it]

fold 5:
{'DT': 0.507, 'RF': 0.59, 'NB': 0.567}
 


{'DT': 0.5176000000000001, 'RF': 0.5955999999999999, 'NB': 0.5751999999999999}

# StopWords Removal + Lemmatization

In [17]:
# Apply Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordlemmatizer = WordNetLemmatizer()

def lemmatizer(text):
    lemm_text = [wordlemmatizer.lemmatize(word) for word in text]
    return lemm_text

lemm = [lemmatizer(x) for x in no_stop]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
# Save the data
#with open("Data/lemm", "wb") as fp:
#    pickle.dump(lemm, fp)

In [5]:
with open("Data/lemm", "rb") as fp:
    lemm = pickle.load(fp)

In [20]:
model = Word2Vec(lemm, min_count=5, vector_size=100, window=7)

In [4]:
train = pd.read_csv('Data/StopWords + Lemm/train.csv', index_col = 1)
test = pd.read_csv('Data/StopWords + Lemm/test.csv', index_col = 1)
data = pd.concat([train,test])

In [24]:
kf = KFold(n_splits=5)

fold = 0

cv = CountVectorizer(binary=True)
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.loc[train_index]
    test = data.loc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['medical_abstract']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['medical_abstract']]))
    
    Kresult = micro_f1(build_results(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB = 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5}
result

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:33<00:00,  6.65s/it]


fold 1:
{'DT': 0.533, 'RF': 0.596, 'NB': 0.569}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:31<00:00,  6.22s/it]


fold 2:
{'DT': 0.519, 'RF': 0.595, 'NB': 0.58}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:32<00:00,  6.42s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:34<00:00,  6.83s/it]


fold 4:
{'DT': 0.511, 'RF': 0.607, 'NB': 0.583}
 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:31<00:00,  6.24s/it]

fold 5:
{'DT': 0.524, 'RF': 0.594, 'NB': 0.57}
 


{'DT': 0.5248, 'RF': 0.5987999999999999, 'NB': 0.5774}

### Evaluating the parameters

In [12]:
for vector_size in [50, 100, 200]:
    for window in [5, 7, 10]:
        model = Word2Vec(lemm, min_count=5, vector_size=vector_size, window=window)
        
        kf = KFold(n_splits=5)
        fold = 0
        Kresults = []

        for train_index, test_index in kf.split(data):
            fold += 1
            train = data.loc[train_index]
            test = data.loc[test_index]

            FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['medical_abstract']]))
            FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['medical_abstract']]))

            Kresult = micro_f1(build_results(FE_train, FE_test, train, test))
            Kresults.append(Kresult)

            DT, RF, NB = 0, 0, 0
            for result in Kresults:
                DT += result['DT']
                RF += result['RF']
                NB += result['NB']
            result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5}
        print('vector_size: ' + str(vector_size) + ', window: ' + str(window))
        print(result)
        print(' ')

vector_size: 50, window: 5
{'DT': 0.5174000000000001, 'RF': 0.5859999999999999, 'NB': 0.5775999999999999}
 
vector_size: 50, window: 7
{'DT': 0.5204000000000001, 'RF': 0.596, 'NB': 0.5835999999999999}
 
vector_size: 50, window: 10
{'DT': 0.5240000000000001, 'RF': 0.5982, 'NB': 0.5921999999999998}
 
vector_size: 100, window: 5
{'DT': 0.52, 'RF': 0.591, 'NB': 0.5755999999999999}
 
vector_size: 100, window: 7
{'DT': 0.522, 'RF': 0.5984, 'NB': 0.5771999999999999}
 
vector_size: 100, window: 10
{'DT': 0.5306, 'RF': 0.6058, 'NB': 0.5833999999999999}
 
vector_size: 200, window: 5
{'DT': 0.5264, 'RF': 0.5964, 'NB': 0.5668}
 
vector_size: 200, window: 7
{'DT': 0.5284000000000001, 'RF': 0.6035999999999999, 'NB': 0.571}
 
vector_size: 200, window: 10
{'DT': 0.536, 'RF': 0.6138, 'NB': 0.5756}
 


### Trying with SVM

In [6]:
model = Word2Vec(lemm, min_count=5, vector_size=200, window=10)

In [7]:
train = pd.read_csv('Data/StopWords + Lemm/train.csv', index_col = 1)
test = pd.read_csv('Data/StopWords + Lemm/test.csv', index_col = 1)
data = pd.concat([train,test])

In [10]:
kf = KFold(n_splits=5)

fold = 0
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.loc[train_index]
    test = data.loc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['medical_abstract']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['medical_abstract']]))
    
    Kresult = micro_f1SVM(build_resultsSVM(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB, SVM = 0, 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
        SVM += result['SVM']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5, 'SVM': SVM/5}
result

fold 1:
{'DT': 0.534, 'RF': 0.61, 'NB': 0.565, 'SVM': 0.627}
 
fold 2:
{'DT': 0.539, 'RF': 0.613, 'NB': 0.581, 'SVM': 0.63}
 
fold 3:
{'DT': 0.544, 'RF': 0.619, 'NB': 0.585, 'SVM': 0.641}
 
fold 4:
{'DT': 0.53, 'RF': 0.614, 'NB': 0.581, 'SVM': 0.639}
 
fold 5:
{'DT': 0.524, 'RF': 0.596, 'NB': 0.567, 'SVM': 0.628}
 


{'DT': 0.5342, 'RF': 0.6104, 'NB': 0.5757999999999999, 'SVM': 0.633}